# Test Parallel ExIFFI

Host capri Davide

    Hostname capri.dei.unipd.it
    User p1026u27

In [61]:
import sys
import numpy as np
import pandas as pd
from tqdm import trange
from append_dir import append_dirname
append_dirname('ExIFFI')
from utils.utils import partition_data
from utils.feature_selection import *
#from plot import *
#from simulation_setup import *
from models import *
from models.Extended_IF import *
from models.Extended_DIFFI_parallel import *
from models.Extended_DIFFI_original import *
import math
import seaborn as sns
sns.set()

from sklearn.preprocessing import StandardScaler
import time

import os
import pickle 
from scipy.io import loadmat
from glob import glob

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)

## Set up file paths

In [15]:
path = os.getcwd()
path = os.path.dirname(path)
path_real = os.path.join(path, "data", "real")
mat_files_real = glob(os.path.join(path_real, "*.mat"))
mat_file_names_real = {os.path.basename(x).split(".")[0]: x for x in mat_files_real}
csv_files_real = glob(os.path.join(path_real, "*.csv"))
csv_file_names_real = {os.path.basename(x).split(".")[0]: x for x in csv_files_real}
dataset_names = list(mat_file_names_real.keys()) + list(csv_file_names_real.keys())
mat_file_names_real.update(csv_file_names_real)
dataset_paths = mat_file_names_real.copy()

## Utility Functions

Drop Duplicates from the loaded dataset 

In [16]:
def drop_duplicates(X, y):
    S = np.c_[X, y]
    S = pd.DataFrame(S).drop_duplicates().to_numpy()
    X, y = S[:, :-1], S[:, -1]
    return X, y

Load dataset coming from a `.mat` file 

In [17]:
def load_data(path):
    data = loadmat(path)
    X, y = data["X"], data["y"]
    y = np.hstack(y)
    X, y = drop_duplicates(X, y)
    return X, y

Load dataset coming from a `.csv` file

In [18]:
def load_data_csv(path):
    data = pd.read_csv(path, index_col=0)
    if "Unnamed: 0" in data.columns:
        data = data.drop(columns=["Unnamed: 0"])

    X = data[data.columns[data.columns != "Target"]]
    y = data["Target"]

    X, y = drop_duplicates(X, y)

    return X, y

Load the data (with `load_data` or with `load_data_csv`), scale the data and split it into train and test set obtaining `X_train`, `X_test` that will be passed to `compute_imps`. 

In [19]:
def pre_process(path):
    extension = os.path.splitext(path)[1]

    if extension == ".csv":
        X, y = load_data_csv(path)
    elif extension == ".mat":
        X, y = load_data(path)
    else:
        raise ValueError("Extension not supported")

    X_train, X_test = partition_data(X, y)
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    X_test = np.r_[X_train, X_test]

    return X_train, X_test

Compute the Global Importance of a given dataset `n_runs` times. At the end a matrix with shape `(n_runs, n_features)` is returned. Each row contains the global importance of the features for a given run.

In [20]:
def compute_imps(model, X_train, X_test, n_runs):

    X_test=np.r_[X_train,X_test]

    imps = np.zeros(shape=(n_runs, X_train.shape[1]))
    for i in tqdm(range(n_runs)):
        model.fit(X_train)
        imps[i, :] = model.Global_importance(
            X_test, calculate=True, overwrite=False, depth_based=False
        )

    return imps

### `test_exiffi`

This is the function called in the `main` of `test_parallel.py` used to do the experiments on the CAPRI HPC server. For a given set of datasets it computes the global importance `n_runs` times using `Extended_DIFFI_parallel` or `Extended_DIFFI_original`and saves the importances matrices, the time stats obtained and the test arguments in a `.npz` file.

#### `test_exiffi` Parameters

- `X_train`: the train set
- `X_test`: the test set
- `savedir`: directory where to save the results in `.npz` format
- `n_runs`: number of runs to do
- `seed`: random seed to obtain reproducibile results and compare the importances matrices obtaind from the parallel and the serial version of the algorithm (they must be the same to certify the correctness of the parallel version)
- `parallel`: Boolean variable used to choose between the parallel and the serial version of the algorithm
- `n_cores`: Number of threads to use in the parallel version of the algorithm. This coincides with the number of cores set with the `--cpus-per-task` options in the `.job` file
- `num_trees`: Number of trees used by ExIFFI. The higher the more complex and more computationally expensive the algorithm is
- `name`: Name of the dataset

In [8]:
def test_exiffi(
    X_train,
    X_test,
    savedir,
    n_runs=10,
    seed=None,
    parallel=False,
    n_cores=2,
    num_trees=300,
    name="",
):
    args_to_avoid = ["X_train", "X_test", "savedir", "args_to_avoid", "args"]
    args = dict()
    for k, v in locals().items():
        if k in args_to_avoid:
            continue
        args[k] = v

    ex_time = []
    ex_imps = {}

    for i in trange(n_runs):
        seed = None if seed is None else seed + i

        if parallel:
            EDIFFI = Extended_DIFFI_parallel(
                n_trees=num_trees, max_depth=100, subsample_size=256, plus=1, seed=seed
            )
            EDIFFI.set_num_processes(n_cores, n_cores)
        else:
            EDIFFI = Extended_DIFFI_original(
                n_trees=num_trees, max_depth=100, subsample_size=256, plus=1, seed=seed
            )

        start = time.time()
        imps = compute_imps(EDIFFI, X_train, X_test, 10)
        ex_imps["Execution " + str(i)] = imps
        end = time.time()
        ex_time.append(end - start)

    # print(ex_imps)
    time_stat = {"mean": np.mean(ex_time), "std": np.std(ex_time)}
    filename = "test_stat_parallel.npz" if parallel else "test_stat_serial.npz"
    t = time.localtime()
    current_time = time.strftime("%d-%m-%Y_%H-%M-%S", t)
    filename = current_time + "_" + name + "_" + filename

    # if dir does not exist, create it
    if not os.path.exists(savedir):
        os.makedirs(savedir)
    filepath = os.path.join(savedir, filename)

    np.savez(
        filepath,
        execution_time_stat=time_stat,
        importances_matrix=ex_imps,
        arguments=args,
    )

## Load Data

## Wine Dataset

In [9]:
name='wine'
X,y=load_data(dataset_paths[name])
X_train,X_test=partition_data(X,y)
X.shape,y.shape

((129, 13), (129,))

### Serial ExIFFI

In [13]:
test_exiffi(
    X_train=X_train,
    X_test=X_test,
    savedir='../results/npz',
    n_runs=1,
    seed=120,
    parallel=False,
    n_cores=12,
    num_trees=10,
    name=name,
)

100%|██████████| 1/1 [00:01<00:00,  2.00s/it]


### Parallel ExIFFI

In [ ]:
test_exiffi(
    X_train=X_train,
    X_test=X_test,
    savedir='../results/npz',
    n_runs=1,
    seed=120,
    parallel=True,
    n_cores=12,
    num_trees=200,
    name=name,
)

## Ionosphere Dataset

In [12]:
name='ionosphere'
X,y=load_data(dataset_paths[name])
X_train,X_test=partition_data(X,y)
X.shape,y.shape

((350, 33), (350,))

### Serial ExIFFI

In [ ]:
test_exiffi(
    X_train=X_train,
    X_test=X_test,
    savedir='../results/npz',
    n_runs=1,
    seed=120,
    parallel=False,
    n_cores=12,
    num_trees=10,
    name=name,
)

### Parallel ExIFFI

In [ ]:
test_exiffi(
    X_train=X_train,
    X_test=X_test,
    savedir='../results/npz',
    n_runs=1,
    seed=120,
    parallel=True,
    n_cores=12,
    num_trees=200,
    name=name,
)

## Moodify Dataset

In [26]:
name='moodify'
X,y=load_data_csv(dataset_paths[name])
X_train,X_test=partition_data(X,y)
X.shape,y.shape

((276260, 11), (276260,))

# Test Results 

## Parallel

In [9]:
stats=np.load('26-01-2024_17-42-35_test_stat_parallel_7000.npz',allow_pickle=True)
data_parallel=stats['importances_matrix'].tolist()
time_data_parallel=stats['execution_time_stat']
arguments_parallel=stats['arguments'].tolist()

In [12]:
print(arguments_parallel.keys())

args_to_avoid = ["X_train", "X_test", "X"]
for key in arguments_parallel.keys():
    if key not in args_to_avoid:
        print(key,arguments_parallel[key])

dict_keys(['X_train', 'X_test', 'X', 'n_runs', 'seed', 'parallel', 'n_cores'])
n_runs 2
seed None
parallel True
n_cores 8


In [33]:
time_data_parallel

array({'mean': 3.36362202167511, 'std': 0.23129910849918273}, dtype=object)

In [34]:
data_parallel.keys()

dict_keys(['Execution 0', 'Execution 1', 'Execution 2', 'Execution 3', 'Execution 4', 'Execution 5', 'Execution 6', 'Execution 7', 'Execution 8', 'Execution 9'])

## Serial

In [3]:
stats=np.load('test_stat_serial.npz',allow_pickle=True)
data_serial=stats['importances_matrix'].tolist()
time_data_serial=stats['execution_time_stat']

FileNotFoundError: [Errno 2] No such file or directory: 'test_stat_serial.npz'

In [30]:
time_data_serial

array({'mean': 3.5636572360992433, 'std': 0.5714168745774968},
      dtype=object)

In [31]:
data_serial.keys()

dict_keys(['Execution 0', 'Execution 1', 'Execution 2', 'Execution 3', 'Execution 4', 'Execution 5', 'Execution 6', 'Execution 7', 'Execution 8', 'Execution 9'])

Check if `data_parallel` and `data_serial` are equal


In [36]:
for k in data_serial.keys():
    print(np.sum(data_serial[k]-data_parallel[k]))

-5.995204332975845e-14
1.532107773982716e-13
-3.4638958368304884e-13
7.327471962526033e-14
5.306866057708248e-13
-3.774758283725532e-13
-3.197442310920451e-13
1.3522516439934407e-12
-1.2434497875801753e-13
-1.0769163338864018e-12


## Results Thyroid

### Parallel

In [9]:
path_to_load = (
    "../capri_code/results/npz/28-01-2024_17-45-18_annthyroid_test_stat_parallel.npz"
)

stats = np.load(path_to_load, allow_pickle=True)

display(stats['execution_time_stat'])
display(stats['arguments'].tolist())

array({'mean': 89.01771640777588, 'std': 0.0}, dtype=object)

{'n_runs': 1,
 'seed': 120,
 'parallel': True,
 'n_cores': 2,
 'num_trees': 10,
 'name': 'annthyroid',
 'args_to_avoid': ['X_train', 'X_test', 'savedir'],
 'args': {...}}

# RESULTS EVALUATION

## Dataset Info

In [97]:
name_samples_features = [
    ("Annthyroid", 7200, 6),
    ("Breastw", 683, 9),
    ("Cardio", 1831, 21),
    ("Glass", 213, 9),
    ("Ionosphere", 351, 33),
    ("Pendigits", 6870, 16),
    ("Pima", 768, 8),
    ("Shuttle", 49097, 9),
    ("Wine", 129, 13),
    ("Diabetes", 85916, 4),
    ("Moodify", 276260, 11),
]

data_name=[]
data_samples=[]
data_feat=[]
data_comp=[]
for n,s,f,comp in size:
    data_name.append(n)
    data_samples.append(s)
    data_feat.append(f)
    data_comp.append(comp)

data_info=pd.DataFrame({
    'Dataset':data_name,
    'Samples':data_samples,
    'Features':data_feat,
    'Complexity':data_comp
})
data_info

,Dataset,Samples,Features,Complexity
0,Wine,129,13,1677
1,Glass,213,9,1917
2,Pima,768,8,6144
3,Breastw,683,9,6147
4,Ionosphere,351,33,11583
5,Cardio,1831,21,38451
6,Annthyroid,7200,6,43200
7,Pendigits,6870,16,109920
8,Diabetes,85916,4,343664
9,Shuttle,49097,9,441873


## All results

We use the script `process_results.py` to read the stats of the experiments from the `.npz` files and display them on a `pd.DataFrame` that can be saved as a `.csv` file.

In [1]:
from append_dir import append_dirname
append_dirname('ExIFFI')

from capri_code.process_results import load_stats, display_stats, compute_cpu_efficiency


results_dirpath = "../../container/job4/results/"

stats = load_stats(results_dirpath, use_pkl=True)

for i, row in stats.iterrows():
    n_cores = max([row["n_cores_fit"], row["n_cores_importance"], row["n_cores_anomaly"]])
    stats.loc[i, "cpu_efficiency"] = compute_cpu_efficiency(row["real_time"], row["user_time"], n_cores)
    

# display_stats(stats)
# display_stats(stats.groupby("parallel").get_group(True))
# display_stats(stats.groupby("parallel").get_group(False))

# sort by "cpu_efficiency"
# display_stats(stats.groupby("parallel").get_group(True).sort_values(by="cpu_efficiency", ascending=False), sort_index=False)
display_stats(stats.sort_values(by="name", ascending=False), sort_index=False)

,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-31 11:17:17.762289,1,1,4,5,120,True,100,wine,5,22.900842,0.509877,339.171738,1.148610,341.471232,117.104,118.939,5.572,25.391746
2024-01-30 19:34:29.794001,4,4,4,5,120,True,300,wine,5,36.091074,6.986208,442.984202,10.987222,450.961408,184.554,411.978,35.911,55.807243
2024-01-30 15:35:08.255912,4,1,1,5,120,True,300,wine,5,36.076435,0.999008,443.505377,11.205296,466.964480,182.849,347.223,10.888,47.474009
2024-01-31 11:22:45.456464,8,8,8,5,120,True,100,wine,5,6.294086,0.157463,343.123231,4.323091,348.299264,33.567,133.018,18.941,49.534513
2024-01-30 15:47:36.088014,1,1,4,5,120,True,300,wine,5,65.135395,1.783640,421.945016,3.925508,428.290048,327.901,340.537,9.254,25.963401
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-01-30 20:38:05.180900,4,4,4,5,120,True,300,annthyroid,5,372.051031,5.036259,573.276488,29.364684,615.780352,1862.890,6965.303,63.040,93.474427
2024-01-31 12:08:12.970003,1,1,1,5,120,False,100,annthyroid,5,461.844206,32.404068,392.240005,2.772541,396.709888,2311.464,2309.838,1.141,99.929655
2024-01-31 11:43:09.387267,4,4,4,5,120,True,100,annthyroid,5,126.100393,2.812973,383.097078,7.249340,391.864320,632.782,2365.817,23.946,93.468880


In [10]:
dataset_names=stats['name'].unique()
dataset_names

array(['cardio', 'pima', 'glass', 'pendigits', 'wine', 'diabetes',
       'moodify', 'annthyroid', 'breastw', 'ionosphere', 'shuttle'],
      dtype=object)

## Results by datset name

In [11]:
for name in dataset_names:
    display_stats(stats.groupby(["name"]).get_group((name)))

/tmp/ipykernel_8918/394651547.py:2: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  display_stats(stats.groupby(["name"]).get_group((name)))


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 15:49:56.673645,1,1,1,5,120,False,100,cardio,5,142.643763,8.523221,393.009725,1.838510,395.968512,716.112,712.554,1.992,99.503150
2024-01-30 16:50:32.736658,4,1,1,5,120,True,300,cardio,5,380.160016,1.962829,736.371671,33.649745,762.687488,1908.615,2271.178,24.292,29.749033
2024-01-30 17:07:52.333205,1,1,1,5,120,False,300,cardio,5,437.642479,18.793363,591.404073,9.619989,608.174080,2190.869,2184.656,5.743,99.716414
2024-01-30 18:01:22.290632,1,1,4,5,120,True,300,cardio,5,387.130552,7.122314,599.861330,5.510218,609.349632,1938.390,2174.331,23.124,28.043002
2024-01-30 18:20:54.680314,4,4,4,5,120,True,300,cardio,5,150.260168,1.874259,633.598607,27.550300,651.247616,757.341,2450.098,59.949,80.878297
2024-01-30 19:31:23.714165,1,1,4,5,120,True,300,cardio,5,399.472970,5.623879,596.438876,5.487201,605.732864,2000.773,2177.073,35.361,27.202899
2024-01-30 19:50:03.422493,4,4,4,5,120,True,300,cardio,5,136.002594,2.305052,632.747950,29.538196,647.999488,683.232,2324.940,65.379,85.071396
2024-01-31 11:25:50.663860,8,8,8,5,120,True,100,cardio,5,26.230811,0.527214,385.819116,5.178153,393.211904,133.281,756.082,28.589,70.910520
2024-01-31 11:26:59.629485,4,4,4,5,120,True,100,cardio,5,44.437300,0.437698,399.578399,14.795008,420.519936,224.730,775.797,23.895,86.303231


/tmp/ipykernel_8918/394651547.py:2: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  display_stats(stats.groupby(["name"]).get_group((name)))


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 17:52:08.295689,1,1,1,5,120,False,100,pima,5,73.023921,1.989698,375.667261,2.361366,378.130432,367.396,366.160,1.081,99.663578
2024-01-30 19:57:12.152079,4,4,4,5,120,True,300,pima,5,85.088809,5.746222,563.433636,22.430954,581.898240,428.836,1292.097,63.086,75.325824
2024-01-31 11:21:31.165166,1,1,1,5,120,False,100,pima,5,73.599623,1.850113,374.940303,1.956828,377.307136,370.370,368.989,1.104,99.627130
2024-01-31 11:27:13.970827,8,8,8,5,120,True,100,pima,5,16.234695,0.146628,379.498988,11.312443,389.828608,84.015,430.135,25.851,63.996757
2024-01-31 11:29:21.224044,4,4,4,5,120,True,100,pima,5,27.788584,0.815569,382.362092,11.641450,396.222464,141.434,445.083,21.908,78.673268
2024-01-31 17:05:15.620974,8,8,8,5,120,True,300,pima,5,50.678049,0.883840,524.159386,26.828166,551.870464,256.204,1287.064,58.299,62.794882


/tmp/ipykernel_8918/394651547.py:2: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  display_stats(stats.groupby(["name"]).get_group((name)))


...

,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 17:56:01.986867,1,1,1,5,120,False,100,breastw,5,46.272488,3.636108,359.480361,1.736259,362.409984,233.615,232.774,0.724,99.640006
2024-01-30 20:01:56.975145,4,4,4,5,120,True,300,breastw,5,56.352648,0.737036,530.465096,16.940998,541.736960,284.412,828.526,52.144,72.827975
2024-01-31 11:25:32.243866,1,1,1,5,120,False,100,breastw,5,47.745507,4.082444,359.160545,1.726413,362.094592,241.007,239.956,0.968,99.563913
2024-01-31 11:28:18.782890,8,8,8,5,120,True,100,breastw,5,11.315375,0.231621,365.879296,7.114440,374.640640,62.605,273.027,24.298,54.513817
2024-01-31 11:30:56.431111,4,4,4,5,120,True,100,breastw,5,18.537070,0.273554,371.985121,14.615467,387.018752,95.158,284.120,19.339,74.644276
2024-01-31 17:08:21.215276,8,8,8,5,120,True,300,breastw,5,36.546677,0.675259,495.119892,15.776448,519.327744,185.184,826.016,51.602,55.756437


/tmp/ipykernel_8918/394651547.py:2: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  display_stats(stats.groupby(["name"]).get_group((name)))


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 18:00:02.972313,1,1,1,5,120,False,100,ionosphere,5,47.742123,0.766028,389.004493,3.439041,394.592256,240.911,239.895,0.872,99.578267
2024-01-30 20:07:02.168474,4,4,4,5,120,True,300,ionosphere,5,60.510995,1.262842,611.246899,39.035752,655.515648,305.277,855.462,59.380,70.056211
2024-01-31 11:29:22.224695,8,8,8,5,120,True,100,ionosphere,5,12.209009,0.255528,384.849019,9.458180,403.341312,63.494,277.242,30.021,54.580354
2024-01-31 11:29:41.333283,1,1,1,5,120,False,100,ionosphere,5,49.343598,0.919404,389.235507,3.402094,394.948608,249.076,247.936,1.004,99.542308
2024-01-31 11:32:36.571651,4,4,4,5,120,True,100,ionosphere,5,19.532191,0.356211,414.830264,21.005797,440.193024,100.022,290.133,23.561,72.517296
2024-01-31 17:11:37.220612,8,8,8,5,120,True,300,ionosphere,5,38.706381,0.507393,578.154988,32.685786,618.020864,195.960,845.564,62.045,53.937283


/tmp/ipykernel_8918/394651547.py:2: FutureWarning: When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.
  display_stats(stats.groupby(["name"]).get_group((name)))


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-31 06:06:48.699817,1,1,1,5,120,False,100,shuttle,5,2847.585005,281.354673,459.580375,2.249807,463.888384,14240.975,14226.500,8.974,99.898357
2024-01-31 13:25:42.392791,8,8,8,5,120,True,100,shuttle,5,455.187845,7.522808,454.499860,26.102526,516.075520,2278.467,14973.973,51.385,82.149385
2024-01-31 14:49:50.046726,4,4,4,5,120,True,100,shuttle,5,779.510358,7.811489,417.636516,17.470480,447.791104,3899.935,14933.852,34.246,95.731416
2024-01-31 22:43:58.637970,8,8,8,5,120,True,300,shuttle,5,1312.437491,29.984434,577.760133,34.560846,622.583808,6566.218,44734.319,83.831,85.159979
2024-01-31 23:23:55.424870,1,1,1,5,120,False,100,shuttle,5,2857.492150,274.462208,462.837514,2.671451,467.120128,14289.911,14281.294,6.119,99.939699


## Serial Results

In [12]:
for name in dataset_names:
    display_stats(stats.groupby(['parallel',"name"]).get_group((False,name)))

,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 15:49:56.673645,1,1,1,5,120,False,100,cardio,5,142.643763,8.523221,393.009725,1.838510,395.968512,716.112,712.554,1.992,99.503150
2024-01-30 17:07:52.333205,1,1,1,5,120,False,300,cardio,5,437.642479,18.793363,591.404073,9.619989,608.174080,2190.869,2184.656,5.743,99.716414


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 17:52:08.295689,1,1,1,5,120,False,100,pima,5,73.023921,1.989698,375.667261,2.361366,378.130432,367.396,366.160,1.081,99.663578
2024-01-31 11:21:31.165166,1,1,1,5,120,False,100,pima,5,73.599623,1.850113,374.940303,1.956828,377.307136,370.370,368.989,1.104,99.627130


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 15:37:59.648176,1,1,1,5,120,False,100,glass,5,38.201773,1.994781,258.565734,48.463303,355.602432,199.292,192.154,1.794,96.418321
2024-01-30 16:31:21.686496,1,1,1,5,120,False,300,glass,5,111.261842,7.624729,470.430024,8.476952,480.997376,558.595,555.479,2.952,99.442172


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 19:15:42.117511,1,1,1,5,120,False,100,pendigits,5,452.142717,17.884384,297.993175,51.293238,403.914752,2263.259,2260.684,1.609,99.886226
2024-01-31 12:45:39.807523,1,1,1,5,120,False,100,pendigits,5,448.885957,16.433706,406.304195,1.145300,408.010752,2247.073,2244.846,1.296,99.900893


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 15:34:38.531939,1,1,1,5,120,False,100,wine,5,22.670529,1.310402,326.859817,7.030727,333.770752,120.767,113.376,0.583,93.879951
2024-01-30 16:22:03.105891,1,1,1,5,120,False,300,wine,5,65.463616,4.144099,404.238664,2.465636,407.486464,329.567,327.989,1.480,99.521190


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-31 02:09:27.569927,1,1,1,5,120,False,100,diabetes,5,4964.569867,193.160756,490.561208,4.016094,497.770496,24825.700,24810.749,9.179,99.939776
2024-01-31 19:25:45.554596,1,1,1,5,120,False,100,diabetes,5,4798.746352,385.346273,488.528937,4.323595,498.876416,24004.258,23985.255,7.064,99.920835


KeyError: (False, 'moodify')

## Parallel Results

In [15]:
for name in dataset_names:
    display_stats(stats.groupby(['parallel',"name"]).get_group((True,name)))

,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 16:50:32.736658,4,1,1,5,120,True,300,cardio,5,380.160016,1.962829,736.371671,33.649745,762.687488,1908.615,2271.178,24.292,29.749033
2024-01-30 18:01:22.290632,1,1,4,5,120,True,300,cardio,5,387.130552,7.122314,599.861330,5.510218,609.349632,1938.390,2174.331,23.124,28.043002
2024-01-30 18:20:54.680314,4,4,4,5,120,True,300,cardio,5,150.260168,1.874259,633.598607,27.550300,651.247616,757.341,2450.098,59.949,80.878297
2024-01-30 19:31:23.714165,1,1,4,5,120,True,300,cardio,5,399.472970,5.623879,596.438876,5.487201,605.732864,2000.773,2177.073,35.361,27.202899
2024-01-30 19:50:03.422493,4,4,4,5,120,True,300,cardio,5,136.002594,2.305052,632.747950,29.538196,647.999488,683.232,2324.940,65.379,85.071396
2024-01-31 11:25:50.663860,8,8,8,5,120,True,100,cardio,5,26.230811,0.527214,385.819116,5.178153,393.211904,133.281,756.082,28.589,70.910520
2024-01-31 11:26:59.629485,4,4,4,5,120,True,100,cardio,5,44.437300,0.437698,399.578399,14.795008,420.519936,224.730,775.797,23.895,86.303231
2024-01-31 17:00:59.660310,8,8,8,5,120,True,300,cardio,5,81.847115,1.072554,580.731372,27.621478,626.106368,411.823,2322.534,65.316,70.495516


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 19:57:12.152079,4,4,4,5,120,True,300,pima,5,85.088809,5.746222,563.433636,22.430954,581.898240,428.836,1292.097,63.086,75.325824
2024-01-31 11:27:13.970827,8,8,8,5,120,True,100,pima,5,16.234695,0.146628,379.498988,11.312443,389.828608,84.015,430.135,25.851,63.996757
2024-01-31 11:29:21.224044,4,4,4,5,120,True,100,pima,5,27.788584,0.815569,382.362092,11.641450,396.222464,141.434,445.083,21.908,78.673268
2024-01-31 17:05:15.620974,8,8,8,5,120,True,300,pima,5,50.678049,0.883840,524.159386,26.828166,551.870464,256.204,1287.064,58.299,62.794882


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 15:40:41.691636,4,1,1,5,120,True,300,glass,5,65.843605,0.978784,421.421711,24.829751,494.878720,331.750,615.674,14.859,46.395931
2024-01-30 15:57:14.231006,1,1,4,5,120,True,300,glass,5,115.173798,1.689243,489.580954,6.211631,500.228096,578.169,602.999,12.623,26.073648
2024-01-30 18:08:17.140138,4,4,4,5,120,True,300,glass,5,50.799305,0.744255,528.016835,21.841332,538.071040,257.075,726.041,41.006,70.605952
2024-01-30 19:38:40.374176,4,4,4,5,120,True,300,glass,5,49.253900,1.624285,517.520425,20.473205,536.113152,248.956,698.099,45.292,70.102649
2024-01-31 11:23:14.895393,4,4,4,5,120,True,100,glass,5,16.065011,0.520735,366.092943,8.140177,375.205888,82.708,236.344,18.349,71.439280
2024-01-31 11:23:37.343592,8,8,8,5,120,True,100,glass,5,9.949837,0.272633,370.483855,8.293028,377.659392,51.870,227.937,23.466,54.929873
2024-01-31 16:54:07.861224,8,8,8,5,120,True,300,glass,5,32.305612,0.659922,481.969439,15.419922,505.745408,164.049,697.476,50.247,53.145402


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 21:11:43.505410,4,4,4,5,120,True,300,pendigits,5,403.024353,39.814432,617.990226,42.473522,653.594624,2018.600,7050.350,81.063,87.317324
2024-01-31 11:42:04.451673,8,8,8,5,120,True,100,pendigits,5,76.724382,0.813368,395.282678,9.007359,408.944640,385.980,2363.045,35.759,76.527443
2024-01-31 11:53:40.183025,4,4,4,5,120,True,100,pendigits,5,125.694113,1.472544,398.152663,11.527904,420.925440,630.867,2340.693,26.828,92.756992
2024-01-31 17:48:20.883909,8,8,8,5,120,True,300,pendigits,5,222.336998,2.295969,560.668836,27.147229,593.149952,1114.180,7036.678,74.978,78.944583


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 15:35:08.255912,4,1,1,5,120,True,300,wine,5,36.076435,0.999008,443.505377,11.205296,466.964480,182.849,347.223,10.888,47.474009
2024-01-30 15:47:36.088014,1,1,4,5,120,True,300,wine,5,65.135395,1.783640,421.945016,3.925508,428.290048,327.901,340.537,9.254,25.963401
2024-01-30 18:04:00.427470,4,4,4,5,120,True,300,wine,5,30.545294,0.439893,442.673889,12.379361,450.551808,155.403,412.897,29.450,66.423589
2024-01-30 19:34:29.794001,4,4,4,5,120,True,300,wine,5,36.091074,6.986208,442.984202,10.987222,450.961408,184.554,411.978,35.911,55.807243
2024-01-31 11:17:17.762289,1,1,4,5,120,True,100,wine,5,22.900842,0.509877,339.171738,1.148610,341.471232,117.104,118.939,5.572,25.391746
2024-01-31 11:21:52.193426,4,4,4,5,120,True,100,wine,5,9.435632,0.411009,339.823493,4.797157,349.523968,49.576,136.592,13.416,68.880103
2024-01-31 11:22:45.456464,8,8,8,5,120,True,100,wine,5,6.294086,0.157463,343.123231,4.323091,348.299264,33.567,133.018,18.941,49.534513
2024-01-31 16:51:23.874894,8,8,8,5,120,True,300,wine,5,19.905357,0.531305,415.790203,9.180716,427.474944,101.985,396.454,37.245,48.592195


...

,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-31 16:49:39.609330,8,8,8,5,120,True,100,moodify,5,2446.791153,33.040259,583.646249,24.667098,606.105600,12237.144,81111.811,211.844,82.854107
2024-01-31 20:42:08.244394,4,4,4,5,120,True,100,moodify,5,4226.958096,71.999307,579.307766,31.618760,615.006208,21138.158,81291.575,233.528,96.143163


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 20:38:05.180900,4,4,4,5,120,True,300,annthyroid,5,372.051031,5.036259,573.276488,29.364684,615.780352,1862.890,6965.303,63.040,93.474427
2024-01-31 11:35:38.451309,8,8,8,5,120,True,100,annthyroid,5,74.788019,1.545447,377.738199,3.589051,382.488576,376.111,2338.203,29.076,77.709872
2024-01-31 11:43:09.387267,4,4,4,5,120,True,100,annthyroid,5,126.100393,2.812973,383.097078,7.249340,391.864320,632.782,2365.817,23.946,93.468880
2024-01-31 17:29:46.391486,8,8,8,5,120,True,300,annthyroid,5,217.337070,2.501002,523.667210,24.805085,556.199936,1089.453,7003.658,60.854,80.357505


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 20:01:56.975145,4,4,4,5,120,True,300,breastw,5,56.352648,0.737036,530.465096,16.940998,541.736960,284.412,828.526,52.144,72.827975
2024-01-31 11:28:18.782890,8,8,8,5,120,True,100,breastw,5,11.315375,0.231621,365.879296,7.114440,374.640640,62.605,273.027,24.298,54.513817
2024-01-31 11:30:56.431111,4,4,4,5,120,True,100,breastw,5,18.537070,0.273554,371.985121,14.615467,387.018752,95.158,284.120,19.339,74.644276
2024-01-31 17:08:21.215276,8,8,8,5,120,True,300,breastw,5,36.546677,0.675259,495.119892,15.776448,519.327744,185.184,826.016,51.602,55.756437


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 20:07:02.168474,4,4,4,5,120,True,300,ionosphere,5,60.510995,1.262842,611.246899,39.035752,655.515648,305.277,855.462,59.380,70.056211
2024-01-31 11:29:22.224695,8,8,8,5,120,True,100,ionosphere,5,12.209009,0.255528,384.849019,9.458180,403.341312,63.494,277.242,30.021,54.580354
2024-01-31 11:32:36.571651,4,4,4,5,120,True,100,ionosphere,5,19.532191,0.356211,414.830264,21.005797,440.193024,100.022,290.133,23.561,72.517296
2024-01-31 17:11:37.220612,8,8,8,5,120,True,300,ionosphere,5,38.706381,0.507393,578.154988,32.685786,618.020864,195.960,845.564,62.045,53.937283


,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-31 13:25:42.392791,8,8,8,5,120,True,100,shuttle,5,455.187845,7.522808,454.499860,26.102526,516.075520,2278.467,14973.973,51.385,82.149385
2024-01-31 14:49:50.046726,4,4,4,5,120,True,100,shuttle,5,779.510358,7.811489,417.636516,17.470480,447.791104,3899.935,14933.852,34.246,95.731416
2024-01-31 22:43:58.637970,8,8,8,5,120,True,300,shuttle,5,1312.437491,29.984434,577.760133,34.560846,622.583808,6566.218,44734.319,83.831,85.159979


## Single Dataset Results

In [29]:
def get_stats_dataset(name,parallel=True,all=False):
    if all:
        df=stats.groupby(["name"]).get_group((name))
        return display_stats(df)
    df=stats.groupby(['parallel',"name"]).get_group((parallel,name))
    return df

In [73]:
def get_stats_compact(df):

    return df

### Wine → (129,13)

In [78]:
df=get_stats_dataset('wine')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
61,wine,100,49.576,136.592,68.880103,4
66,wine,300,155.403,412.897,66.423589,4
8,wine,300,184.554,411.978,55.807243,4
21,wine,100,33.567,133.018,49.534513,8
34,wine,300,101.985,396.454,48.592195,8
38,wine,300,182.849,347.223,47.474009,4
17,wine,300,327.901,340.537,25.963401,4
16,wine,100,117.104,118.939,25.391746,4


Here we can notice how the CPU efficiency is not very high because `wine` is a very small and "easy" dataset. In any case we can notice that the efficiency is higher for the case of 4 cores. THe three cases with 4 cores with a low efficiency are the ones where not all the paralelizable parts of the code are parallelized (so one between `n_cores_fit`,`n_cores_importance` and `n_cores_anomaly` are set to 1). 

On the other hand we can see this counterintuitive effect where the CPU Efficiency is smaller when we increase the number of cores from 4 to 8. One of the reasons for that may be that increasing the number of cores we are increasing the overhead needed to organize the threads. 

### Glass → (213,9)

In [79]:
df=get_stats_dataset('glass')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
62,glass,100,82.708,236.344,71.439280,4
48,glass,300,257.075,726.041,70.605952,4
12,glass,300,248.956,698.099,70.102649,4
52,glass,100,51.870,227.937,54.929873,8
40,glass,300,164.049,697.476,53.145402,8
69,glass,300,331.750,615.674,46.395931,4
2,glass,300,578.169,602.999,26.073648,4


Similar to `wine` but the efficiency increases a little

In [134]:
257/60,248/60

(4.283333333333333, 4.133333333333334)

### Pima → (768,8)

In [88]:
df=get_stats_dataset('pima')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
29,pima,100,141.434,445.083,78.673268,4
64,pima,300,428.836,1292.097,75.325824,4
1,pima,100,84.015,430.135,63.996757,8
63,pima,300,256.204,1287.064,62.794882,8


The number of samples increases a lot with the respect to `wine` and `glass` so also the efficiency increased by a bit

### Breastw → (683,9)

In [87]:
df=get_stats_dataset('breastw')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
22,breastw,100,95.158,284.120,74.644276,4
57,breastw,300,284.412,828.526,72.827975,4
24,breastw,300,185.184,826.016,55.756437,8
44,breastw,100,62.605,273.027,54.513817,8


Similar to `pima`

### Ionosphere → (351,33)

In [86]:
df=get_stats_dataset('ionosphere')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
49,ionosphere,100,100.022,290.133,72.517296,4
23,ionosphere,300,305.277,855.462,70.056211,4
56,ionosphere,100,63.494,277.242,54.580354,8
43,ionosphere,300,195.960,845.564,53.937283,8


Similar to `pima` and `breastw`. Here it seems that the number of features (that is very high in this dataset) is not affecting the execution time. It seems that the main computational effort comes when there are a lot of samples. 

### Cardio → (1831,21)

In [89]:
df=get_stats_dataset('cardio')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
9,cardio,100,224.730,775.797,86.303231,4
15,cardio,300,683.232,2324.940,85.071396,4
47,cardio,300,757.341,2450.098,80.878297,4
31,cardio,100,133.281,756.082,70.910520,8
10,cardio,300,411.823,2322.534,70.495516,8
26,cardio,300,1908.615,2271.178,29.749033,4
41,cardio,300,1938.390,2174.331,28.043002,4
13,cardio,300,2000.773,2177.073,27.202899,4


Here with 4 cores fully parallel we get to an higher than 80% of CPU Efficiency

### Annthyroid → (7200,6)

In [84]:
df=get_stats_dataset('annthyroid')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
27,annthyroid,300,1862.890,6965.303,93.474427,4
25,annthyroid,100,632.782,2365.817,93.468880,4
51,annthyroid,300,1089.453,7003.658,80.357505,8
39,annthyroid,100,376.111,2338.203,77.709872,8


The number of samples increased a lot here and so also the efficiency which is higher than 90% 

### Pendigits → (6870,16)

In [83]:
df=get_stats_dataset('pendigits')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
58,pendigits,100,630.867,2340.693,92.756992,4
3,pendigits,300,2018.600,7050.350,87.317324,4
33,pendigits,300,1114.180,7036.678,78.944583,8
53,pendigits,100,385.980,2363.045,76.527443,8


Similar to `annthyroid`

### Diabetes → (85916,4)

In [82]:
df=get_stats_dataset('diabetes')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
65,diabetes,100,6669.827,25660.739,96.182176,4
32,diabetes,300,19753.084,75955.915,96.131717,4
46,diabetes,300,11171.858,77912.916,87.175423,8
54,diabetes,100,3939.523,25865.904,82.071814,8


Here we increased of one order of magnitude so the efficiency is higher than 80% both with 4 and 8 cores

### Shuttle → (49097,9)

In [81]:
df=get_stats_dataset('shuttle')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
55,shuttle,100,3899.935,14933.852,95.731416,4
68,shuttle,300,6566.218,44734.319,85.159979,8
71,shuttle,100,2278.467,14973.973,82.149385,8


Similar to `shuttle`

### Moodify → (276260,11)

In [80]:
df=get_stats_dataset('moodify')
df=df[['name','n_trees','real_time','user_time','cpu_efficiency','n_cores_fit','n_cores_importance','n_cores_anomaly']]
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
df.drop(columns=['n_cores_fit','n_cores_importance','n_cores_anomaly'],inplace=True)
df.sort_values(by='cpu_efficiency',ascending=False)

,name,n_trees,real_time,user_time,cpu_efficiency,n_cores
7,moodify,100,21138.158,81291.575,96.143163,4
19,moodify,100,12237.144,81111.811,82.854107,8


`Moodify` is a very big dataset and it takes a while to work with it so for the moment we were able to produce just the results with the parallel algorithm which obviously have a very high CPU Efficiency

## Compute Speedup and Efficiency

In [105]:
results_dirpath = "../../container/job4/results/"

stats = load_stats(results_dirpath, use_pkl=True)

for i, row in stats.iterrows():
    n_cores = max([row["n_cores_fit"], row["n_cores_importance"], row["n_cores_anomaly"]])
    stats.loc[i, "cpu_efficiency"] = compute_cpu_efficiency(row["real_time"], row["user_time"], n_cores)

### Wine

In [109]:
display_stats(stats.groupby('name').get_group('wine'))

,n_cores_fit,n_cores_importance,n_cores_anomaly,n_runs,seed,parallel,n_trees,name,n_runs_imps,mean_time,std_time,mean_MB,std_MB,max_MB,real_time,user_time,sys_time,cpu_efficiency
time,,,,,,,,,,,,,,,,,,
2024-01-30 15:34:38.531939,1,1,1,5,120,False,100,wine,5,22.670529,1.310402,326.859817,7.030727,333.770752,120.767,113.376,0.583,93.879951
2024-01-30 15:35:08.255912,4,1,1,5,120,True,300,wine,5,36.076435,0.999008,443.505377,11.205296,466.964480,182.849,347.223,10.888,47.474009
2024-01-30 15:47:36.088014,1,1,4,5,120,True,300,wine,5,65.135395,1.783640,421.945016,3.925508,428.290048,327.901,340.537,9.254,25.963401
2024-01-30 16:22:03.105891,1,1,1,5,120,False,300,wine,5,65.463616,4.144099,404.238664,2.465636,407.486464,329.567,327.989,1.480,99.521190
2024-01-30 18:04:00.427470,4,4,4,5,120,True,300,wine,5,30.545294,0.439893,442.673889,12.379361,450.551808,155.403,412.897,29.450,66.423589
2024-01-30 19:34:29.794001,4,4,4,5,120,True,300,wine,5,36.091074,6.986208,442.984202,10.987222,450.961408,184.554,411.978,35.911,55.807243
2024-01-31 11:17:17.762289,1,1,4,5,120,True,100,wine,5,22.900842,0.509877,339.171738,1.148610,341.471232,117.104,118.939,5.572,25.391746
2024-01-31 11:21:52.193426,4,4,4,5,120,True,100,wine,5,9.435632,0.411009,339.823493,4.797157,349.523968,49.576,136.592,13.416,68.880103
2024-01-31 11:22:45.456464,8,8,8,5,120,True,100,wine,5,6.294086,0.157463,343.123231,4.323091,348.299264,33.567,133.018,18.941,49.534513


### 100 trees

In [126]:
df=stats.groupby(['name',"n_trees"]).get_group(('wine',100))
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
serial_time=float(df[df['parallel']==False]['real_time'])
df['Speedup']=df['real_time']/serial_time
df['1/Speedup']=serial_time/df['real_time']
df['Efficiency']=df['Speedup']/df['n_cores']
df=df[['name','n_trees','n_cores','Speedup','1/Speedup','Efficiency']]
df

/tmp/ipykernel_8918/1665553493.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
/tmp/ipykernel_8918/1665553493.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Speedup']=df['real_time']/serial_time
/tmp/ipykernel_8918/1665553493.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

,name,n_trees,n_cores,Speedup,1/Speedup,Efficiency
4,wine,100,1,1.000000,1.000000,1.000000
16,wine,100,4,0.969669,1.031280,0.242417
21,wine,100,8,0.277948,3.597789,0.034744
61,wine,100,4,0.410509,2.435997,0.102627


The easier conclusion here is that with 8 cores we are more than 3 times faster than the serial version and with 4 cores we are more than 2 times faster. 

### 300 trees

In [127]:
df=stats.groupby(['name',"n_trees"]).get_group(('wine',300))
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
serial_time=float(df[df['parallel']==False]['real_time'])
df['Speedup']=df['real_time']/serial_time
df['1/Speedup']=serial_time/df['real_time']
df['Efficiency']=df['Speedup']/df['n_cores']
df=df[['name','n_trees','n_cores','Speedup','1/Speedup','Efficiency']]
df

/tmp/ipykernel_8918/1396246683.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
/tmp/ipykernel_8918/1396246683.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Speedup']=df['real_time']/serial_time
/tmp/ipykernel_8918/1396246683.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

,name,n_trees,n_cores,Speedup,1/Speedup,Efficiency
8,wine,300,4,0.559989,1.785748,0.139997
17,wine,300,4,0.994945,1.005081,0.248736
34,wine,300,8,0.309451,3.231524,0.038681
38,wine,300,4,0.554816,1.802400,0.138704
60,wine,300,1,1.000000,1.000000,1.000000
66,wine,300,4,0.471537,2.120725,0.117884


### Glass

#### 100 trees

In [133]:
df=stats.groupby(['name','n_trees']).get_group(('glass',100))
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
serial_time=float(df[df['parallel']==False]['real_time'])
df['Speedup']=df['real_time']/serial_time
df['1/Speedup']=serial_time/df['real_time']
df['Efficiency']=df['Speedup']/df['n_cores']
df=df[['name','n_trees','n_cores','Speedup','1/Speedup','Efficiency']]
df

/tmp/ipykernel_8918/3052569487.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
/tmp/ipykernel_8918/3052569487.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Speedup']=df['real_time']/serial_time
/tmp/ipykernel_8918/3052569487.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value in

,name,n_trees,n_cores,Speedup,1/Speedup,Efficiency
18,glass,100,1,1.000000,1.000000,1.000000
52,glass,100,8,0.260271,3.842144,0.032534
62,glass,100,4,0.415009,2.409586,0.103752


#### 300 trees

In [132]:
df=stats.groupby(['name','n_trees']).get_group(('glass',300))
df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
serial_time=float(df[df['parallel']==False]['real_time'])
df['Speedup']=df['real_time']/serial_time
df['1/Speedup']=serial_time/df['real_time']
df['Efficiency']=df['Speedup']/df['n_cores']
df=df[['name','n_trees','n_cores','Speedup','1/Speedup','Efficiency']]
df

/tmp/ipykernel_8918/544743139.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['n_cores']=[np.max(np.array([i,j,k])) for i,j,k in zip(df['n_cores_fit'],df['n_cores_importance'],df['n_cores_anomaly'])]
/tmp/ipykernel_8918/544743139.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Speedup']=df['real_time']/serial_time
/tmp/ipykernel_8918/544743139.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inste

,name,n_trees,n_cores,Speedup,1/Speedup,Efficiency
2,glass,300,4,1.035041,0.966145,0.258760
5,glass,300,1,1.000000,1.000000,1.000000
12,glass,300,4,0.445682,2.243750,0.111421
40,glass,300,8,0.293681,3.405050,0.036710
48,glass,300,4,0.460217,2.172887,0.115054
69,glass,300,4,0.593901,1.683783,0.148475
